# Notebook to look at linearity

Initially written 31 Aug 2020 by Craig Lage.\
Using pkl'd data so I can run it on my laptop.


In [ ]:
import sys, os, glob, time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
from scipy.optimize import fmin_powell
import scipy.interpolate as interp
import astropy.io.fits as pf

In [ ]:
DATA_DIR = '/project/shared/BOT/'
REPO_DIR = '/project/cslage/BOT_lspdev/E2V_6790D_Gain_Mu12_R22S11/'
RUN = '6790D'
[RAFT, SENSOR, DETECTOR] = ['R22', 'S11', 94]
#[RAFT, SENSOR, DETECTOR] = ['R02', 'S02', 11]

In [ ]:
file = open('/Users/cslage/Research/LSST/code/BOT/gain/linearized/pickle/linearity_r22s11_c04.pkl','rb')
#file = open('/project/cslage/BOT_lspdev/ptcs/linearity_r22s11_c04.pkl','rb')
pkl_data = pkl.load(file)
file.close()
print(pkl_data.keys())
rawMeans = pkl_data['rawMeans']
bothRawMeans = pkl_data['bothRawMeans']
headerData = pkl_data['headerData']
rawVars = pkl_data['rawVars']
means = pkl_data['means']
expTimes = pkl_data['expTimes']
gains = pkl_data['gains']
pairs = pkl_data['pairs']

In [ ]:
print(xx[300:305], yy[300:305])

In [ ]:
flux_max_e = 150000  # electrons

amp = 'C04'
flux_max = flux_max_e / gains[amp] # In ADU

index = expTimes.argsort()
means = means[index]
expTimes = expTimes[index]

lin_index = np.where(means < 20000)
coefs = np.polyfit(means[lin_index], expTimes[lin_index], 1)
slope = coefs[0]

lin_model = slope * means

lin_residuals = (expTimes - lin_model) / expTimes * 100.0

xplot = np.linspace(100.0,flux_max, 200)
yplot = np.ones_like(xplot)

knots = 10
xx = means
yy = expTimes / slope

# Force the spline to match the linear fit at low flux by adding 
# highly weighted points at low flux
xx_points = np.array([0.0,500.0,5000.0,20000.0])
xx = np.insert(xx, 0, xx_points)
yy = np.insert(yy, 0, xx_points)
weights = np.ones_like(xx)
weights[0:len(xx_points)] = 100000.0

index = xx. argsort()
xx = xx[index]
yy = yy[index]
weights = weights[index]

length = xx[-1]-xx[0]
t = np.linspace(xx[0]+1e-5*length, xx[-1]-1e-5*length, knots)
print("Input knots",t[1:-2])
s = interp.splrep(xx, yy, w=weights, task=-1, t=t[1:-2])
print("Output knots",s[0])
s_model = interp.splev(means, s) 
s_yplot = interp.splev(xplot, s)
s_residuals = ((means / expTimes * slope) - s_model) * 100.0

fig = plt.figure(figsize = (16,16))
plt.suptitle("Flux Nonlinearity - Run 6790D, R22S11 C04 28Aug2020", fontsize = 24)
plt.subplot(2,1,1)
plt.plot(xplot, yplot, color='green', label = 'Low flux linear fit')
plt.plot(xplot, s_yplot / xplot, color='blue', label = '%d knot spline fit'%knots)
#plt.scatter(means, means/expTimes, marker = 'x', s=100, color='red')
plt.scatter(xx, yy / xx, marker = 'x', s=100, color='red')
plt.ylabel("Flux/ExpTime*LowFluxSlope", fontsize = 18)
plt.xlabel("Flux(ADU)", fontsize = 18)
plt.legend(fontsize=18)
plt.ylim(0.995, 1.005)
#plt.xscale('log')

plt.subplot(2,1,2)
plt.scatter(means[1:-1],lin_residuals[1:-1], label = 'Linear Residuals')
plt.scatter(means[1:-1],s_residuals[1:-1], label = 'Spline Residuals')
plt.plot([0.0, 120000.0], [0.0,0.0], ls = '--', color = 'black')
plt.ylabel("Residuals (%)", fontsize = 18)
plt.xlabel("Flux(ADU)", fontsize = 18)

#plt.xscale('log')
plt.legend(fontsize = 18)
#plt.savefig(REPO_DIR + 'plots/Flux_Correction_R22S11_C04_28Aug20.pdf')

In [ ]:
f = 50000
print(interp.splev(f, s))
sder = interp.splder(s)
print(interp.splev(f,sder)**2)
print(rvars[200])
print(0.814, (interp.splev(f*0.814,sder)**2)*f*0.814 / interp.splev(f, s))
print(these_coefs[1] + these_coefs[0] * 51732)

In [ ]:
amp = 'C04'
sder = interp.splder(s)
rvars = np.array(rawVars[amp])
rmeans = np.array(rawMeans[amp])
correctedMeans = interp.splev(rmeans, s)
correctedVars = (interp.splev(rvars, sder))**2 * rvars
fig = plt.figure(figsize = (16,8))
plt.title("Non-linearity, R22S11, C04", fontsize = 24)
cut_means = []
cut_vars = []
# Fit a line to the points outside the bump
for i, mean in enumerate(rmeans):
    if (mean >10000 and mean <25000) or (mean >65000 and mean <100000):
        cut_means.append(mean)
        cut_vars.append(rvars[i]/mean)
    #if mean > 50000:
    #    print(mean, rvars[i]/mean)
    #    break
xplot = np.linspace(-5000,120000, 100)
these_coefs = np.polyfit(cut_means, cut_vars, 1)
yplot = these_coefs[1] + these_coefs[0] * xplot
print()
plt.scatter(rmeans,rvars/rmeans, color = 'red', marker = 'o', label = 'UnCorrected')
plt.scatter(rmeans,correctedVars/correctedMeans, color = 'green', marker = 'x', s = 100,label = 'Corrected')
plt.plot(xplot, yplot, ls = '--', color = 'blue')
plt.xlabel("Flux(ADU)", fontsize = 18)
plt.ylabel("Variance / Flux (ADU)", fontsize = 18)
plt.ylim(0.6, 1.1)
plt.legend(fontsize = 18)
#plt.savefig(REPO_DIR + 'plots/NonLinearity_R22S11_C04_28Aug20.pdf')